In [0]:
# 1. Tell Git about yourself:
from getpass import getpass
u = input('Username: ') # Git username
p = getpass('Password: ') # Git password

Username: mdmeldon
Password: ··········


In [0]:
# 2. Initialize Git, then Pull the Branch:
! git init
! git remote add origin https://$u:$p@github.com/mdmeldon/DS-CV.git 
! git pull origin master

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 161 (delta 54), reused 125 (delta 25), pack-reused 0
Receiving objects: 100% (161/161), 2.59 MiB | 2.50 MiB/s, done.
Resolving deltas: 100% (54/54), done.
From https://github.com/mdmeldon/DS-CV
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


In [0]:
# 3. Run helper:
! sh colab.sh

     |████████████████████████████████| 337kB 2.8MB/s 
     |████████████████████████████████| 368kB 7.8MB/s 
     |████████████████████████████████| 552kB 13.8MB/s 
     |████████████████████████████████| 460kB 18.5MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 1.8MB 18.5MB/s 
     |████████████████████████████████| 276kB 30.2MB/s 
     |████████████████████████████████| 235kB 30.4MB/s 
     |████████████████████████████████| 112kB 24.5MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 204kB 25.1MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 552kB 20.6MB/s 
     |████████████████████████████████| 2.7MB 32.9MB/s 
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=2727abe0cd2adee58c7433fbeb8c16

In [0]:
! dvc pull

Estimating size of cache in 'gdrive://1fX3XdpFjdrNbc2QE0.00 [00:00,     ?file/s]Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/0QHexgxctuhVU3BlFYrcA9i7goQyaSregOfBuvXftMBtB6oXyNI1800
Authentication successful.
!

!



!





!







!





                                                                                


                                                                                
                                                                                



                                                                                




6 added


In [0]:
# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#import torchvision
# from models.resnet import resnet50
from metrics.auc import AUCCallback
import torchvision
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst.dl import SupervisedRunner
from catalyst.dl.utils import set_global_seed, prepare_cudnn
from catalyst.dl.callbacks import AccuracyCallback, PrecisionRecallF1ScoreCallback, VerboseLogger, ConfusionMatrixCallback

# Other  
from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import sys
import IPython.display as ipd  # To play sound in the notebook
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



Я использую 4 датасета, а именно:

    1)SAVEE dataset
        Этот набор данных только из мужчин и имеет очень высокое качество 
        звука. Поскольку на записи только мужчины, это приведет к некоторому 
        дисбалансу, было бы целесообразно дополнить другие наборы данных 
        женскими голосами.

    2)RAVDESS dataset
      Является одним из наиболее распространенных наборов данных, используемых  
      для распознавания эмоций другими. Он мне очень понравился из-за его 
      качества выступающих, хорошей записи, и в нем 24 актера разных полов. 

    3)TESS dataset
        Основан только на двух спикерах, молодой женщине и пожилой женщине. 
        Надеюсь, что это сбалансирует доминирующих мужчин-спикеров из SAVEE.


    4)CREMA-D dataset
        Это очень большой набор данных у которого есть большое разнообразие     
        разных записей, по-видимому, взятых из фильмов. Спикеры разных 
        национальностей, а значит лучшее мы имеем возможность тестировать
        выявлекние эмоций вне рамок языка.


Все 4 набора данных являются хорошими наборами данных. Чтобы дать моей модели возможность хорошо работать с новыми наборами данных, я объединяю эти четыре датасета в один. Создав шум и объединив 4 различных набора данных, связанных между собой только идеей создать датасет с эмоциями, я надеюсь найти реальные отличительные характеристики для выявления эмоций.

In [0]:
import pandas as pd
ref = pd.read_csv("data/processed/Data_path.csv")
print(ref.labels.value_counts())
ref.head()

female_fear       1096
female_happy      1096
female_angry      1096
female_disgust    1096
female_sad        1096
female_neutral    1056
male_neutral       839
male_fear          827
male_angry         827
male_happy         827
male_disgust       827
male_sad           827
Name: labels, dtype: int64


,labels,source,path
0,male_disgust,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
1,male_disgust,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
2,male_sad,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
3,male_neutral,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...
4,male_fear,SAVEE,../data/raw/surrey-audiovisual-expressed-emoti...


In [0]:
'''
1. Data Augmentation method   
'''
def speedNpitch(data):
    """
    Speed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change # try changing 1.0 to 2.0 ... =D
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

'''
2. Extracting the MFCC feature as an image (Matrix format).  
'''
def feature_extraction(df, n, aug, mfcc_b):
    X = np.empty(shape=(df.shape[0], n, 216, 1))
    input_length = sampling_rate * audio_duration
    
    cnt = 0
    for fname in tqdm(df.path):
        file_path = fname
        data, _ = librosa.load(file_path, sr=sampling_rate
                               ,res_type="kaiser_fast"
                               ,duration=2.5
                               ,offset=0.5
                              )

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")

        # Augmentation? 
        if aug == 1:
            data = speedNpitch(data)
        
        # which feature?

        if mfcc_b == 1:
            # MFCC extraction 
            MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X[cnt,] = MFCC
            
        else:
            # Log-melspectogram
            melspec = librosa.feature.melspectrogram(data, n_mels = n_melspec)   
            logspec = librosa.amplitude_to_db(melspec)
            logspec = np.expand_dims(logspec, axis=-1)
            X[cnt,] = logspec
            
            
        cnt += 1
    
    return X


In [0]:
# from src.features.build_features import feature_extraction
# sampling_rate=44100
# audio_duration=2.5
# n_mfcc = 30
# mfcc1 = feature_extraction(df= ref, n_mfcc= n_mfcc, aug = 0, sampling_rate= sampling_rate, audio_duration= audio_duration)

In [0]:
# np.save('../data/processed/MFCC_PREPARE.npy', mfcc)
# mfcc = np.load('data/processed/.npy')

In [0]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(mfcc,
                                                    ref.labels,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=8
                                                   )


In [0]:
X_test.shape

(2878, 30, 216, 1)

In [0]:
# one hot encode the target 
lb = LabelEncoder()

y_train = lb.fit_transform(y_train) # to_categorical
y_test = lb.fit_transform(y_test) # to_categorical

# Normalization as per the standard NN process
# mean = np.mean(X_train, axis=0)
# std = np.std(X_train, axis=0)

# X_train = (X_train - mean)/std
# X_test = (X_test - mean)/std

#! Normalization as per the standard NN process
from sklearn.preprocessing import StandardScaler
def fit_X_scaler(X_train):
    """fit StandardScaler，and return StandardScaler object
    """
    sc = StandardScaler()
    for _, clips in enumerate(X_train):
        data_i_truncated = np.squeeze(clips)
        sc.partial_fit(data_i_truncated)
    return sc

def get_X_scaled(X_train, scaler=None):
    """apply normlization
    """
    X_train_new = np.zeros(X_train.shape)
    for indx, clips in enumerate(X_train):
        data_i_truncated = np.squeeze(clips)
        if scaler is not None:  # normlize
            data_i_truncated = scaler.transform(data_i_truncated)
        X_train[indx, :, :, 0] = data_i_truncated
    return X_train

sc = fit_X_scaler(X_train)
X_train = get_X_scaled(X_train, sc)
X_test = get_X_scaled(X_test, sc)

In [0]:
classes_names = list(lb.classes_)
dict_classes_names = { i : classes_names[i] for i in range(0, len(classes_names) ) }
dict_classes_names

{0: 'female_angry',
 1: 'female_disgust',
 2: 'female_fear',
 3: 'female_happy',
 4: 'female_neutral',
 5: 'female_sad',
 6: 'male_angry',
 7: 'male_disgust',
 8: 'male_fear',
 9: 'male_happy',
 10: 'male_neutral',
 11: 'male_sad'}

In [0]:
# model = resnet50(pretrained=False, progress=True, num_classes=12)

In [0]:
model = torchvision.models.resnet50(pretrained=True,progress=True)

In [0]:
# Изменение количества выходных классов 
n_class = 12
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, n_class)

In [0]:
# Изменение количества входных слоёв
model.conv1 = torch.nn.Conv2d(30, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)

In [0]:
BATCH_SIZE = 16

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.FloatTensor(X_train[0:-1])
targetsTrain = torch.from_numpy(y_train[0:-1]).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.FloatTensor(X_test[0:-1])
targetsTest = torch.from_numpy(y_test[0:-1]).type(torch.LongTensor) # data type is long

trainSet = TensorDataset(featuresTrain,targetsTrain)
validSet = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(trainSet, batch_size = BATCH_SIZE, shuffle = True)
valid_loader = DataLoader(validSet, batch_size = BATCH_SIZE, shuffle = False)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [0]:
# Init catalyst components
runner = SupervisedRunner(device=device)
loaders = {'train': train_loader, 'valid': valid_loader}
logdir = 'data/logs/1'
callbacks = [
             AUCCallback(num_classes = 12, class_names= classes_names),
             AccuracyCallback(),
             ConfusionMatrixCallback(num_classes = 12, class_names= classes_names),

    #AUCCallback(num_classes = 14),
    # VerboseLogger(),
    # PrecisionRecallF1ScoreCallback(num_classes = 14),
]

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir data/logs/1

In [0]:
runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        callbacks=callbacks,
        logdir=logdir,
        num_epochs=120,
        verbose=True,        
    )  


1/120 * Epoch (train):   0% 0/540 [00:00<?, ?it/s]
1/120 * Epoch (train):   0% 0/540 [00:00<?, ?it/s, accuracy01=0.188, loss=2.335]
1/120 * Epoch (train):   0% 1/540 [00:00<01:23,  6.44it/s, accuracy01=0.188, loss=2.335]
1/120 * Epoch (train):   0% 1/540 [00:00<01:23,  6.44it/s, accuracy01=0.562, loss=1.580]
1/120 * Epoch (train):   0% 2/540 [00:00<01:24,  6.34it/s, accuracy01=0.562, loss=1.580]
1/120 * Epoch (train):   0% 2/540 [00:00<01:24,  6.34it/s, accuracy01=0.375, loss=1.904]
1/120 * Epoch (train):   1% 3/540 [00:00<01:22,  6.47it/s, accuracy01=0.375, loss=1.904]
1/120 * Epoch (train):   1% 3/540 [00:00<01:22,  6.47it/s, accuracy01=0.250, loss=2.219]
1/120 * Epoch (train):   1% 4/540 [00:00<01:20,  6.63it/s, accuracy01=0.250, loss=2.219]
1/120 * Epoch (train):   1% 4/540 [00:00<01:20,  6.63it/s, accuracy01=0.188, loss=2.265]
1/120 * Epoch (train):   1% 5/540 [00:00<01:20,  6.68it/s, accuracy01=0.188, loss=2.265]
1/120 * Epoch (train):   1% 5/540 [00:00<01:20,  6.68it/s, accurac

In [0]:
! zip -r logs.zip data/logs/

  adding: data/logs/ (stored 0%)
  adding: data/logs/0/ (stored 0%)
  adding: data/logs/0/checkpoints/ (stored 0%)
  adding: data/logs/0/checkpoints/_metrics.json (deflated 76%)
  adding: data/logs/0/checkpoints/train.7.pth (deflated 17%)
  adding: data/logs/0/checkpoints/last.pth (deflated 16%)
  adding: data/logs/0/checkpoints/last_full.pth (deflated 28%)
  adding: data/logs/0/checkpoints/train.7_full.pth (deflated 28%)
  adding: data/logs/0/checkpoints/best_full.pth (deflated 28%)
  adding: data/logs/0/checkpoints/best.pth (deflated 17%)
  adding: data/logs/0/_base_log/ (stored 0%)
  adding: data/logs/0/_base_log/events.out.tfevents.1590755877.70706919b3ea.120.0 (deflated 68%)
  adding: data/logs/0/valid_log/ (stored 0%)
  adding: data/logs/0/valid_log/events.out.tfevents.1590755973.70706919b3ea.120.2 (deflated 31%)
  adding: data/logs/0/train_log/ (stored 0%)
  adding: data/logs/0/train_log/events.out.tfevents.1590755877.70706919b3ea.120.1 (deflated 47%)
  adding: data/logs/0/log.t